In [2]:
import pandas as pd
from sklearn.preprocessing import *
from sklearn_pandas import DataFrameMapper
import numpy as np
import matplotlib.pyplot as plt
import re
df_train = pd.read_csv('/home/mmaesawy/Desktop/Work/Kaggle-Housing/train.csv')
df_test = pd.read_csv('/home/mmaesawy/Desktop/Work/Kaggle-Housing/test.csv')
df_full = pd.concat([df_train, df_test], axis = 0)

In [4]:
print('Missing values per column:')
for c in range(df_full.shape[1]):
    missing_count = df_full.iloc[:, c].shape[0] - df_full.iloc[:, c].count()  
    if missing_count > 0:
        print(f'{c}:\t{df_full.columns[c]}'
              f' - Missing: {missing_count}'
              f' - # Unique values: {len(pd.value_counts(df_full.iloc[:, c]))}')


Missing values per column:
3:	Alley - Missing: 2721 - # Unique values: 2
6:	BsmtCond - Missing: 82 - # Unique values: 4
7:	BsmtExposure - Missing: 82 - # Unique values: 4
8:	BsmtFinSF1 - Missing: 1 - # Unique values: 991
9:	BsmtFinSF2 - Missing: 1 - # Unique values: 272
10:	BsmtFinType1 - Missing: 79 - # Unique values: 6
11:	BsmtFinType2 - Missing: 80 - # Unique values: 6
12:	BsmtFullBath - Missing: 2 - # Unique values: 4
13:	BsmtHalfBath - Missing: 2 - # Unique values: 3
14:	BsmtQual - Missing: 81 - # Unique values: 4
15:	BsmtUnfSF - Missing: 1 - # Unique values: 1135
19:	Electrical - Missing: 1 - # Unique values: 5
23:	Exterior1st - Missing: 1 - # Unique values: 15
24:	Exterior2nd - Missing: 1 - # Unique values: 16
25:	Fence - Missing: 2348 - # Unique values: 4
26:	FireplaceQu - Missing: 1420 - # Unique values: 5
30:	Functional - Missing: 2 - # Unique values: 7
31:	GarageArea - Missing: 1 - # Unique values: 603
32:	GarageCars - Missing: 1 - # Unique values: 6
33:	GarageCond - Missing

76:	Utilities - Missing: 2 - # Unique values: 2


There exists a lot of missing values. However, reading the data description, it's clear that the data is very clean and that the missing values are real values not incomplete data. We replace all np.nan values with a placeholder below, as to not confuse scikit-learn's algorithms. Also, we replace the np.nan values for contineous features with a value of zero, which is more appropriate for these features.

In [ ]:
p = re.compile(r'$((SF[0-9]*)|(Area)|(Frontage))')
for c in range(df_full.shape[1]):
    if p.search(df_full.columns[c]):
        df_full.iloc[:, c].fillna(0, inplace = True)

missing_label = 'None'
df_full[[x for x in df_full.columns if not x == 'SalePrice']].fillna(missing_label,
                                                                     inplace = True)

We then examine the relation between each continuous feature and the target variable in order to get a better feel for the data. We define a contin

In [ ]:
cont_feature = []
for c in range(df_full.shape[1]):
    if len(pd.value_counts(df_full.iloc[:, c])) > 100:
        cont_feature.append(df_full.columns[c])
        
plt.ion()
i = 0
while i < len(cont_feature):
    fig, axs = plt.subplots(nrows = 4,
                       ncols= 4,
                       sharex = 'col',
                       sharey = 'row')
    axs = axs.flatten()
    for ax in axs:
        if i < len(cont_feature): break
        if cont_feature[i] != 'SalePrice':
            ax.scatter(x = df_full[cont_feature[i]], y = df_full['SalePrice'])
        else:
            ax.hist(df_train[cont_feature[i]]) # Used df_train to avoid errors due to NaNs
        ax.set_xlabel(cont_feature[i])
        ax.set_ylabel('SalePrice')
        i += 1
    fig.show()

/home/mmaesawy/anaconda3/lib/python3.6/site-packages/matplotlib/figure.py:403: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "


/home/mmaesawy/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
